# December 20, 2023

https://adventofcode.com/2023/day/20

In [355]:
import re
from collections import defaultdict
import math

In [2]:
test1 = f'''broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a'''

test2 = f'''broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output'''

test1_text = test1.split("\n")
test2_text = test2.split("\n")

In [30]:
tmp['a']

0

In [32]:
tmp = defaultdict( lambda: "low" )
if "low" in tmp.values():
    print("yo")

In [3]:
test1_text

['broadcaster -> a, b, c', '%a -> b', '%b -> c', '%c -> inv', '&inv -> a']

In [191]:
fn = "data/20.txt"
with open(fn, "r") as file:
    text = file.readlines()

puzz_text = [x.strip() for x in text]

In [333]:
class Module:
    def __init__(self, name, receivers):
        self.name = name
        self.receivers = receivers.split(", ")
        self.receivers = ["rx" if x == "output" else x for x in self.receivers]

    def receive(self, pulse):
        raise NotImplementedError()
    
    def send(self, pulse):
        return [ [self.name, r, pulse] for r in self.receivers]
    
    def receivers(self):
        return self.receivers
    
    def __repr__(self):
        return self.__str__()
           
    
class FlipFlop(Module):
    def __init__(self, name, receivers):
        super().__init__(name, receivers)
        self.type = "flipflop"
        self.status = "off"

    def receive(self, sender, pulse):
        if pulse == "low":
            if self.status == "off":
                self.status = "on"
                return self.send( "high" )
            else:
                self.status = "off"
                return self.send( "low" )

    def __str__(self):
        return f'''%{self.name} <{self.status}> -> {", ".join(self.receivers)}'''



class Conjunction(Module):
    def __init__(self, name, receivers):
        super().__init__(name, receivers)
        self.type = "conjunction"
        self.status = "off"
        self.senders = dict()

    def add_sender(self, sender):
        self.senders[sender] = "low"

    def receive(self, sender, pulse):
        self.senders[sender] = pulse
        if "low" in self.senders.values():
            return self.send("high")
        else:
            return self.send("low")

    def __str__(self):
        return f'''&{self.name} {{{", ".join([str(n)+":"+str(s) for n,s in self.senders.items()])}}} -> {", ".join(self.receivers)}'''

class Broadcaster(Module):
    def __init__(self, name, receivers):
        super().__init__(name, receivers)
        self.type = "broadcaster"
        self.status = 0

    def receive(self, sender, pulse):
        return self.send(pulse)

    def __str__(self):
        return f'''broadcaster -> {", ".join(self.receivers)}'''

class Button(Module):
    def __init__(self):
        super().__init__("button", "broadcaster")
        self.type = "button"
        self.status = 0

    def receive(self, pulse=None):
        return self.send("low")
    
    def __str__(self):
        return "button -> broadcaster"
    
class System:
    def __init__(self, text, part2=False):
        self.modules = {"button":Button()}
        self.pushes = 0
        self.pulse_counts = {"low":0, "high":0}
        self.activated = False
        self.part2 = part2
        # part1 systems are always valid, part2 require an rx module
        self.valid_system = not part2
        
        # create all the modules
        for line in text:
            name, out = line.split(" -> ")
            if name == "broadcaster":
                # move broadcaster to top
                sub = Broadcaster
            else:
                pref = name[0]
                name = name[1:]
                if pref == "%":
                    sub = FlipFlop
                elif pref == "&":
                    sub = Conjunction
                else:
                    raise Exception("unknown module prefix", pref)
            self.modules[name] = sub(name, out)

        # initialize inputs for conjunctions
        for name, mod in self.modules.items():
            for rcv in mod.receivers:
                if rcv == "rx":
                    self.valid_system = True
                if rcv in self.modules.keys() and self.modules[rcv].type == "conjunction":
                    self.modules[rcv].add_sender(name)

        if not self.valid_system:
            raise Exception("part2 systems require connection to 'rx' module")
        
        # determine critical modules for part2 system
        # list of times each of these components gets a LOW signal and flips status
        self.critical = {}
        if self.part2:
            for name, mod in self.modules.items():
                if "rx" in mod.receivers:
                    # assuming that the module that triggers rx is a conjunction, not a FlipFlop
                    # assuming all inputs to it are conjunctions with one input 
                    for sender in mod.senders:
                        self.critical[sender] = []
                        if self.modules[sender].type != "conjunction":
                            raise Exception("need to implement critical flipflop modules")
                        elif len( self.modules[sender].senders ) > 1:
                            raise Exception("need to implement critical conjunction modules with multiple senders")

    
    def push(self):
        self.pushes += 1
        pulse_queue = [ ["button", "broadcaster", "low"] ]
        while len(pulse_queue) > 0:
            pulse = pulse_queue[0]
            sender = pulse[0]
            receiver = pulse[1]
            level = pulse[2]

            #print( f'''{sender} -{level}-> {receiver}''')
            self.pulse_counts[level] += 1
            if self.part2:
                if receiver in self.critical.keys():
                    if level == "low":
                        self.critical[receiver].append( self.pushes )

                if receiver == "rx" and level == "low":
                    # end state!
                    self.activated = True

            if receiver in self.modules.keys():
                effect = self.modules[ receiver ].receive( sender, level ) or []
            else:
                effect = []
                
            pulse_queue = pulse_queue[1:] + effect
            
        return self.activated
                
    def __repr__(self):
        return self.__str__()

    def __str__(self):
        mod_str = "\n".join( [str(x) for x in self.modules.values()] )
        return f'''Modules:\n{mod_str}\nLow Pulses: {self.pulse_counts["low"]}\nHigh Pulses: {self.pulse_counts["high"]}'''


In [334]:
test = System(test1_text)

In [335]:
test

Modules:
button -> broadcaster
broadcaster -> a, b, c
%a <off> -> b
%b <off> -> c
%c <off> -> inv
&inv {c:low} -> a
Low Pulses: 0
High Pulses: 0

In [336]:
test = System(test1_text)
print("status")
print(test)
print("\n\n")
test.push()
print("\nstatus")
print(test)

status
Modules:
button -> broadcaster
broadcaster -> a, b, c
%a <off> -> b
%b <off> -> c
%c <off> -> inv
&inv {c:low} -> a
Low Pulses: 0
High Pulses: 0




status
Modules:
button -> broadcaster
broadcaster -> a, b, c
%a <off> -> b
%b <off> -> c
%c <off> -> inv
&inv {c:low} -> a
Low Pulses: 8
High Pulses: 4


In [337]:
test2 = System(test2_text)
test2

Modules:
button -> broadcaster
broadcaster -> a
%a <off> -> inv, con
&inv {a:low} -> b
%b <off> -> con
&con {a:low, b:low} -> rx
Low Pulses: 0
High Pulses: 0

In [338]:
test2.push()

test2

Modules:
button -> broadcaster
broadcaster -> a
%a <on> -> inv, con
&inv {a:high} -> b
%b <on> -> con
&con {a:high, b:high} -> rx
Low Pulses: 4
High Pulses: 4

In [339]:
test2.push()

test2

Modules:
button -> broadcaster
broadcaster -> a
%a <off> -> inv, con
&inv {a:low} -> b
%b <on> -> con
&con {a:low, b:high} -> rx
Low Pulses: 8
High Pulses: 6

In [340]:
test2.push()

test2

Modules:
button -> broadcaster
broadcaster -> a
%a <on> -> inv, con
&inv {a:high} -> b
%b <off> -> con
&con {a:high, b:low} -> rx
Low Pulses: 13
High Pulses: 9

In [341]:
test2.push()

test2

Modules:
button -> broadcaster
broadcaster -> a
%a <off> -> inv, con
&inv {a:low} -> b
%b <off> -> con
&con {a:low, b:low} -> rx
Low Pulses: 17
High Pulses: 11

### Part 1

In [342]:
def part1( puzz ):
    system = System(puzz)
    for i in range(1000):
        system.push()

    pc = system.pulse_counts
    ans = pc["low"] * pc["high"]

    return ans, system

In [343]:
ans, sys = part1( test1_text )
print(ans, sys, sep="\n\n")

32000000

Modules:
button -> broadcaster
broadcaster -> a, b, c
%a <off> -> b
%b <off> -> c
%c <off> -> inv
&inv {c:low} -> a
Low Pulses: 8000
High Pulses: 4000


In [344]:
ans, sys = part1( test2_text )
print(ans, sys, sep="\n\n")

11687500

Modules:
button -> broadcaster
broadcaster -> a
%a <off> -> inv, con
&inv {a:low} -> b
%b <off> -> con
&con {a:low, b:low} -> rx
Low Pulses: 4250
High Pulses: 2750


In [345]:
ans, sys = part1( puzz_text )
ans

825167435

### Part 2

In [309]:
# naive solution takes too long!
def part2( puzz ):
    system = System(puzz, part2 = True)
    bn_status = system.modules["bn"].senders.copy()
    #bn_status = 100
    print(bn_status)

    while not system.activated:
        system.push()
        bn_update = system.modules["bn"].senders

        if bn_update != bn_status:
            print(f'''After {system.pushes} Pushes\n{system.modules['bn']}''')
            bn_status = bn_update.copy()

        if system.pushes % 100000 == 0:
            print( f'''Push: {system.pushes}\tLow: {system.pulse_counts["low"]}\tHigh: {system.pulse_counts["high"]}''')

    print(system)
    return system.pushes

In [346]:
puzz = System(puzz_text, part2=True)

In [347]:
puzz.critical

{'pl': [], 'mz': [], 'lz': [], 'zm': []}

### Second solution

Determine when the critical modules receive a low signal
That means they'll send a high signal to the module that triggers rx

It seems all of the critical modules receive a high signal if they don't receive low
Therefore, we need the first time they all receive a low signal.

In [348]:
for i in range(100000):
    puzz.push()

In [349]:
puzz.critical

{'pl': [3797,
  7594,
  11391,
  15188,
  18985,
  22782,
  26579,
  30376,
  34173,
  37970,
  41767,
  45564,
  49361,
  53158,
  56955,
  60752,
  64549,
  68346,
  72143,
  75940,
  79737,
  83534,
  87331,
  91128,
  94925,
  98722],
 'mz': [3881,
  7762,
  11643,
  15524,
  19405,
  23286,
  27167,
  31048,
  34929,
  38810,
  42691,
  46572,
  50453,
  54334,
  58215,
  62096,
  65977,
  69858,
  73739,
  77620,
  81501,
  85382,
  89263,
  93144,
  97025],
 'lz': [4003,
  8006,
  12009,
  16012,
  20015,
  24018,
  28021,
  32024,
  36027,
  40030,
  44033,
  48036,
  52039,
  56042,
  60045,
  64048,
  68051,
  72054,
  76057,
  80060,
  84063,
  88066,
  92069,
  96072],
 'zm': [3823,
  7646,
  11469,
  15292,
  19115,
  22938,
  26761,
  30584,
  34407,
  38230,
  42053,
  45876,
  49699,
  53522,
  57345,
  61168,
  64991,
  68814,
  72637,
  76460,
  80283,
  84106,
  87929,
  91752,
  95575,
  99398]}

In [352]:
for k, v in puzz.critical.items():
    delta = [ x1-x0 for x1,x0 in zip(v, [0, *v[:-1]]) ]
    print(delta)

[3797, 3797, 3797, 3797, 3797, 3797, 3797, 3797, 3797, 3797, 3797, 3797, 3797, 3797, 3797, 3797, 3797, 3797, 3797, 3797, 3797, 3797, 3797, 3797, 3797, 3797]
[3881, 3881, 3881, 3881, 3881, 3881, 3881, 3881, 3881, 3881, 3881, 3881, 3881, 3881, 3881, 3881, 3881, 3881, 3881, 3881, 3881, 3881, 3881, 3881, 3881]
[4003, 4003, 4003, 4003, 4003, 4003, 4003, 4003, 4003, 4003, 4003, 4003, 4003, 4003, 4003, 4003, 4003, 4003, 4003, 4003, 4003, 4003, 4003, 4003]
[3823, 3823, 3823, 3823, 3823, 3823, 3823, 3823, 3823, 3823, 3823, 3823, 3823, 3823, 3823, 3823, 3823, 3823, 3823, 3823, 3823, 3823, 3823, 3823, 3823, 3823]


In [364]:
def factorize( arr ):
    factors = []
    for x in arr:
        factors[x] = set()
        for i in range(1,x+1):
            if x/i == int(x/i):
                factors[x].append(i)

    return factors

In [365]:
lcm([3797, 3881, 4003, 3823])

{3797: {1, 3797}, 3881: {1, 3881}, 4003: {1, 4003}, 3823: {1, 3823}}

In [366]:
# all loops are prime, so lcm is just the product
3797*3881*4003*3823

225514321828633

In [289]:
class System2(System):
    def __init__( self, text ):
        self.__graph = None
        super().__init__( text, part2 = True )

    def graph( self ):
        if self.__graph is None:
            self.graph_upstream()
        return self.__graph
    
    def graph_upstream( self ):
        self.__graph = defaultdict( set )

        done = False
        while not done:
            done = True
            print("loop")
            for mod in self.modules.values():
                print(mod)
                for rcv in mod.receivers:
                    old_len = len( self.__graph[rcv] )
                    self.__graph[rcv].add( mod.name )
                    self.__graph[rcv] = self.__graph[rcv].union( self.__graph[mod.name] )
                    new_len = len( self.__graph[rcv] )
                    if new_len != old_len:
                        done = False
            

In [306]:
test = System2( test2_text )

In [307]:
test.graph()

loop
button -> broadcaster
broadcaster -> a
%a <off> -> inv, con
&inv {a:low} -> b
%b <off> -> con
&con {a:low, b:low} -> rx
loop
button -> broadcaster
broadcaster -> a
%a <off> -> inv, con
&inv {a:low} -> b
%b <off> -> con
&con {a:low, b:low} -> rx


defaultdict(set,
            {'broadcaster': {'button'},
             'button': set(),
             'a': {'broadcaster', 'button'},
             'inv': {'a', 'broadcaster', 'button'},
             'con': {'a', 'b', 'broadcaster', 'button', 'inv'},
             'b': {'a', 'broadcaster', 'button', 'inv'},
             'rx': {'a', 'b', 'broadcaster', 'button', 'con', 'inv'}})

In [308]:
puzz = System2(puzz_text)
puzz.graph()

loop
button -> broadcaster
%cf <off> -> hl, qt
&bn {pl:low, mz:low, lz:low, zm:low} -> rx
%nb <off> -> vt
%hm <off> -> jp
%vr <off> -> qt, sl
%gq <off> -> hm, nl
%sl <off> -> jx, qt
&pl {qt:low} -> bn
%hf <off> -> vt, ch
%kx <off> -> dq
%fr <off> -> qf
%rh <off> -> vr
&vt {nb:low, hf:low, hn:low, ch:low, kd:low, cb:low, hh:low, kr:low} -> lz, dh, kr, kq, lm, qk
&dq {kx:low, lv:low, gx:low, mh:low, xd:low, mt:low, ts:low} -> mz, ml, xd, fb, xs, rc, rt
%hn <off> -> qk, vt
%bv <off> -> nl
%jv <off> -> rh, qt
%kq <off> -> lm
%nd <off> -> hp
%gj <off> -> bv, nl
%lv <off> -> xs, dq
%ch <off> -> vt, kd
%sm <off> -> qt, nd
%nt <off> -> jv
%qk <off> -> cb
%jx <off> -> cf
%hl <off> -> qt, ng
&qt {cf:low, vr:low, sl:low, jv:low, sm:low, hl:low, hp:low, ng:low} -> sm, rh, nd, jx, nt, pl
%bh <off> -> nl, fr
%kd <off> -> vt, nb
%gx <off> -> mh, dq
%hp <off> -> nt, qt
%rc <off> -> lv
broadcaster -> kr, zb, sm, xd
&mz {dq:low} -> bn
%qf <off> -> rd, nl
%sk <off> -> nl, bh
%rb <off> -> nl, sk
%cb <off>

defaultdict(set,
            {'broadcaster': {'button'},
             'button': set(),
             'hl': {'broadcaster',
              'button',
              'cf',
              'hl',
              'hp',
              'jv',
              'jx',
              'nd',
              'ng',
              'nt',
              'qt',
              'rh',
              'sl',
              'sm',
              'vr'},
             'cf': {'broadcaster',
              'button',
              'cf',
              'hl',
              'hp',
              'jv',
              'jx',
              'nd',
              'ng',
              'nt',
              'qt',
              'rh',
              'sl',
              'sm',
              'vr'},
             'qt': {'broadcaster',
              'button',
              'cf',
              'hl',
              'hp',
              'jv',
              'jx',
              'nd',
              'ng',
              'nt',
              'qt',
              'rh',
              

In [321]:
puzz = System(puzz_text)
puzz.modules['broadcaster'].receivers = ["kr"]

In [326]:
puzz = System(puzz_text)
puzz.modules['broadcaster'].receivers = ["kr"]

for i in range(1,4098):
    print(i)
    puzz.push()

    if i >= 4096 or i <= 16:
        print(puzz)


1
button -low-> broadcaster
broadcaster -low-> kr
kr -high-> hh
kr -high-> vt
vt -high-> lz
vt -high-> dh
vt -high-> kr
vt -high-> kq
vt -high-> lm
vt -high-> qk
lz -low-> bn
bn -high-> rx
Modules:
button -> broadcaster
%cf <off> -> hl, qt
&bn {pl:low, mz:low, lz:low, zm:low} -> rx
%nb <off> -> vt
%hm <off> -> jp
%vr <off> -> qt, sl
%gq <off> -> hm, nl
%sl <off> -> jx, qt
&pl {qt:low} -> bn
%hf <off> -> vt, ch
%kx <off> -> dq
%fr <off> -> qf
%rh <off> -> vr
&vt {nb:low, hf:low, hn:low, ch:low, kd:low, cb:low, hh:low, kr:high} -> lz, dh, kr, kq, lm, qk
&dq {kx:low, lv:low, gx:low, mh:low, xd:low, mt:low, ts:low} -> mz, ml, xd, fb, xs, rc, rt
%hn <off> -> qk, vt
%bv <off> -> nl
%jv <off> -> rh, qt
%kq <off> -> lm
%nd <off> -> hp
%gj <off> -> bv, nl
%lv <off> -> xs, dq
%ch <off> -> vt, kd
%sm <off> -> qt, nd
%nt <off> -> jv
%qk <off> -> cb
%jx <off> -> cf
%hl <off> -> qt, ng
&qt {cf:low, vr:low, sl:low, jv:low, sm:low, hl:low, hp:low, ng:low} -> sm, rh, nd, jx, nt, pl
%bh <off> -> nl, fr


In [318]:
puzz.push()

button -low-> broadcaster
broadcaster -low-> kr
kr -low-> hh
kr -low-> vt
hh -high-> vt
hh -high-> dh
vt -high-> lz
vt -high-> dh
vt -high-> kr
vt -high-> kq
vt -high-> lm
vt -high-> qk
vt -high-> lz
vt -high-> dh
vt -high-> kr
vt -high-> kq
vt -high-> lm
vt -high-> qk
lz -low-> bn
lz -low-> bn
bn -high-> rx
bn -high-> rx


False

In [319]:
puzz.push()

button -low-> broadcaster
broadcaster -low-> kr
kr -high-> hh
kr -high-> vt
vt -high-> lz
vt -high-> dh
vt -high-> kr
vt -high-> kq
vt -high-> lm
vt -high-> qk
lz -low-> bn
bn -high-> rx


False

In [320]:
puzz.push()

button -low-> broadcaster
broadcaster -low-> kr
kr -low-> hh
kr -low-> vt
hh -low-> vt
hh -low-> dh
vt -high-> lz
vt -high-> dh
vt -high-> kr
vt -high-> kq
vt -high-> lm
vt -high-> qk
vt -high-> lz
vt -high-> dh
vt -high-> kr
vt -high-> kq
vt -high-> lm
vt -high-> qk
dh -high-> kq
lz -low-> bn
lz -low-> bn
bn -high-> rx
bn -high-> rx


False